## Visualize prediction results
Feeds randomly selected images into a trained model and visualizes the results.
Follow this [guide](https://towardsdatascience.com/running-jupyter-notebook-in-google-cloud-platform-in-15-min-61e16da34d52) for setting up jupyter notebooks with gcloud compute.

### Setup
Steps to configure the prediction visualizer.

In [ ]:
%matplotlib inline

# Create symlinks so can import them here
!ln -s ../scripts/draw_grid.py draw_grid.py
!ln -s ../src/imgcls/config config
!ln -s ../src/imgcls/modeling modeling

import os
import json
import random
import numpy as np
from matplotlib import pyplot as plt
from draw_grid import draw_pred_vs_true
from config import get_cfg
import modeling
from detectron2.engine import DefaultPredictor
import cv2
import torch

#### Specify paths
Specify the relative file paths for the data, config file, and model weights.

In [ ]:
# Specify paths
data_json = '../data/full/frames_100/val.json'  # train.json, val.json or test.json
config_file = '../output/X-101-grid-4-thres-0.15/config.yaml'
model_weights = '../output/X-101-grid-4-thres-0.15/model_best.pth'

#### Set parameters
Set the grid size and the min/max image sizes to feed to the model.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(config_file)
cfg.MODEL.WEIGHTS = os.path.abspath(model_weights)
cfg.INPUT.FORMAT = "BGR" # this is the case when using cv2.imread()

# Specify grid size
GRID_SIZE = 4

# specify min/max image sizes
cfg.INPUT.MIN_SIZE_TEST = 224
cfg.INPUT.MAX_SIZE_TEST = 224

# use default predictor, docs below
# https://detectron2.readthedocs.io/modules/engine.html#detectron2.engine.defaults.DefaultPredictor
predictor = DefaultPredictor(cfg)

with open(data_json, 'r') as json_file:
    dataset_dicts = json.load(json_file)

#### Define method
Main method for generating predictions for a random sample from dataset.

In [ ]:
def visualize_preds():
    """Gets a random sample from 'data_json' feeds it into the model and compares predictions to truth labels"""
    d = random.sample(dataset_dicts, 1)[0]
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    plt.imshow(img[:, :, ::-1])  # convert image from OpenCV BGR format to Matplotlib RGB format
    plt.show()
    output = predictor(img)
    predictions = output['pred'].cpu().round().numpy().astype(int)
    truth_labels = np.array(d['label'])
    print("labels: " + str(truth_labels))
    print("preds:  " + str(predictions))
    img = draw_pred_vs_true(img,
                            predictions.reshape(GRID_SIZE, GRID_SIZE), 
                            truth_labels.reshape(GRID_SIZE, GRID_SIZE), GRID_SIZE)
   
    plt.imshow(img)
    plt.show()

#### Main
This is the main code block which displays the predictions and truth labels. Run this block several times to get a
qualitative sense of how well the model is doing

In [ ]:
visualize_preds()

#### Clean up symlinks
Once done with notebook, run this code block to remove the symlinks.

In [ ]:
!rm draw_grid.py modeling config ../src/imgcls/config/config ../src/imgcls/modeling/modeling